In [252]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.svm import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.metrics import *
from sklearn.impute import *
from sklearn.multioutput import *
from sklearn.ensemble import *

In [253]:
with open("../data/processed/train.pkl", "rb") as f:
    train = pickle.load(f)

In [254]:
with open("../data/processed/target.pkl", "rb") as f:
    target = pickle.load(f)

In [255]:
train.head()

,Пол,Семья,Этнос,Национальность,Религия,Образование,Профессия,Вы работаете?,Выход на пенсию,Прекращение работы по болезни,...,Частота пасс кур,Алкоголь,Возраст алког,Время засыпания,Время пробуждения,Сон после обеда,"Спорт, клубы","Религия, клубы",Продолжительность сна,Образование_ord
ID,,,,,,,,,,,,,,,,,,,,,
54-102-358-02,М,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,...,NaN,употребляю в настоящее время,18.0,22:00:00,06:00:00,0,0,0,8,3
54-103-101-01,Ж,в разводе,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,...,NaN,никогда не употреблял,NaN,00:00:00,04:00:00,1,0,0,4,5
54-501-026-03,Ж,в браке в настоящее время,европейская,Русские,Христианство,5 - ВУЗ,дипломированные специалисты,0,0,0,...,1.5,употребляю в настоящее время,17.0,23:00:00,07:00:00,0,0,0,8,5
54-501-094-02,М,в браке в настоящее время,европейская,Русские,Атеист / агностик,3 - средняя школа / закон.среднее / выше среднего,низкоквалифицированные работники,1,0,0,...,4.5,употребляю в настоящее время,13.0,23:00:00,07:00:00,0,0,0,8,3
54-503-022-01,Ж,в браке в настоящее время,европейская,Русские,Христианство,3 - средняя школа / закон.среднее / выше среднего,операторы и монтажники установок и машинного о...,0,0,1,...,7.0,употребляю в настоящее время,16.0,23:00:00,06:00:00,0,0,0,7,3


In [256]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 955 entries, 54-102-358-02 to 54-702-016-01
Data columns (total 34 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   Пол                                    955 non-null    category
 1   Семья                                  955 non-null    category
 2   Этнос                                  955 non-null    category
 3   Национальность                         955 non-null    category
 4   Религия                                955 non-null    category
 5   Образование                            955 non-null    category
 6   Профессия                              955 non-null    category
 7   Вы работаете?                          955 non-null    int8    
 8   Выход на пенсию                        955 non-null    int8    
 9   Прекращение работы по болезни          955 non-null    int8    
 10  Сахарный диабет                        955 no

In [257]:
target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 955 entries, 54-102-358-02 to 54-702-016-01
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   Артериальная гипертензия            955 non-null    int8 
 1   ОНМК                                955 non-null    int8 
 2   Стенокардия, ИБС, инфаркт миокарда  955 non-null    int8 
 3   Сердечная недостаточность           955 non-null    int8 
 4   Прочие заболевания сердца           955 non-null    int8 
dtypes: int8(5)
memory usage: 12.1+ KB


In [258]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 'Сердечная недостаточность', 'Прочие заболевания сердца']
ID_COL = 'ID'
EDU_COL = 'Образование'
SEX_COL = 'Пол'
CAT_COLS = [
    'Пол', 'Семья', 'Этнос', 'Национальность', 'Религия', 'Образование', 
    'Профессия', 'Статус Курения', 'Алкоголь',
    'Время засыпания', 'Время пробуждения'
]
OHE_COLS = [
    'Вы работаете?', 'Выход на пенсию', 'Прекращение работы по болезни', 'Сахарный диабет', 'Гепатит',
    'Онкология', 'Хроническое заболевание легких', 'Бронжиальная астма', 'Туберкулез легких ', 'ВИЧ/СПИД',
    'Регулярный прим лекарственных средств', 'Травмы за год', 'Переломы','Пассивное курение', 'Сон после обеда', 
    'Спорт, клубы', 'Религия, клубы'
]
REAL_COLS = ['Возраст курения', 'Сигарет в день', 'Возраст алког', 'Частота пасс кур']

In [259]:
train[CAT_COLS] = train[CAT_COLS].astype('object')

In [260]:
train_data, val_data, train_target, val_target = train_test_split(train, target, train_size=0.8, random_state=42)

In [261]:
real_pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
]
)

In [262]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [263]:
import category_encoders as ce

In [264]:
preprocess_pipe = ColumnTransformer(transformers=[
    ('real_cols', real_pipe, REAL_COLS),
    ('cat_cols', cat_pipe, CAT_COLS),
    ('woe_cat_cols', ce.WOEEncoder(), CAT_COLS),
    ('ohe_cols', 'passthrough', OHE_COLS)
]
)

In [265]:
model = LinearSVC()

In [266]:
model_pipe = Pipeline([
    ('preprocess', preprocess_pipe),
    ('model', model)
]
)

In [267]:
multiout_model_pipe = MultiOutputClassifier(model_pipe, n_jobs=4)

In [268]:
from catboost import CatBoostClassifier

In [269]:
cat_boost_model = CatBoostClassifier(loss_function='MultiLogloss',
                # custom_metric=['Recall', 'F1'],
                iterations=2000,
                silent=True,
                cat_features=CAT_COLS)

In [270]:
stack = StackingClassifier([("catboost", cat_boost_model), ("sk model", model_pipe)])
stack = MultiOutputClassifier(stack, n_jobs=4)

In [277]:
scores = cross_val_score(
    estimator=stack,
    X=train,
    y=target,
    scoring='f1_samples',
    cv=3,
    n_jobs=1
)

/home/ebuzanov/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ebuzanov/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ebuzanov/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ebuzanov/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ebuzanov/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:1098: RuntimeWarning: Number of classes in training fold (1) does not match total number of classes (2). Results may not be appropriate for your use case. To fix this, use a cross-validation tech

ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/home/ebuzanov/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 428, in _process_worker
    r = call_item()
  File "/home/ebuzanov/.local/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 275, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/home/ebuzanov/.local/lib/python3.8/site-packages/joblib/_parallel_backends.py", line 620, in __call__
    return self.func(*args, **kwargs)
  File "/home/ebuzanov/.local/lib/python3.8/site-packages/joblib/parallel.py", line 288, in __call__
    return [func

In [278]:
scores

NameError: name 'scores' is not defined